In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import pathlib

In [81]:
driver = webdriver.Firefox()
driver.implicitly_wait(10)
url = "https://www.ebay.de/sch/i.html?_from=R40&_nkw=blackmagic+pocket+cinema+camera+4k&_sacat=0&LH_TitleDesc=0&_fsrp=1&LH_Complete=1&LH_Sold=1&Marke=Blackmagic%7CBlackmagic%2520Design&_dcat=11724&rt=nc&LH_PrefLoc=3" 
driver.get(url)
# Click on the coockies preference banner after some time to be sure the element is clickable
time.sleep(2)
driver.find_element(By.ID, "gdpr-banner-accept").click()
# Get the page source and get the content of it with BeautifulSoup
page_source = driver.page_source
soup = BeautifulSoup(page_source)

# We want to retrieve the results of the search, which are stored in an 
#unordered-list (ul) html item and we specify a selector ("class") to choose the right elem.
results = soup.find("ul", {"class" : "srp-results srp-list clearfix"})
#results.text

In [84]:
#The text in previous output isn't helpful. 
# We want to process each item of the unordered list, more than having the whole output 
#in a single string:
items = results.findAll("li")
for item in items:
    #print(item)
    itemBreak = item.find("li", {"class" : "srp-river-answer"})
    print(itemBreak)
    itemTitle = item.find("div", {"class" : "s-item__title"})
    itemPrice = item.find("span", {"class": "s-item__price"})
    itemDate = item.find("span", {"class" : "POSITIVE"}) #POSITIVE=sold, NEGATIVE=ended (no bids)
    # There are some itemTitle None. Exclude them
    if itemTitle:
        # Delete the annoying "NEW LISTING" Tag:
        newListingTag = itemTitle.find("span", {"class" : "LIGHT_HIGHLIGHT"})
        if newListingTag:
            newListingTag.extract()
        itemTitle = itemTitle.text
        print(itemTitle)
    if itemPrice:
        itemPrice = itemPrice.text
        itemPrice = float(itemPrice.split()[-1].replace(".", "").replace(",","."))
        print(itemPrice)
    if itemDate:
        # Eliminate "verkauft" from the date
        itemDate = itemDate.text
        itemDate = itemDate.split()[1:]
        # HARDCODED: Change Dez to Dec
        itemDate[1] = "Dec" if itemDate[1] == "Dez" else itemDate[1]
        # Apply date formatting %b %d, %Y
        itemDate = itemDate[1]+" "+itemDate[0][:-1]+", "+itemDate[2]
        # Create a datetime object to store the date
        itemDate = datetime.strptime(itemDate, "%b %d, %Y")
        print(itemDate.strftime("%B %d, %Y"))
        
    if "Ergebnisse für weniger Suchbegriffe" in item.strings:
        break

None
None
None
None
None
None
None
None
BLACKMAGIC DESIGN POCKET CINEMA CAMERA 4K BLACK MAGIC BMPCC4K Fast Neuwertig
942.0
February 08, 2023
None
Blackmagic Design 4K 120fps Kamera - SET
1050.0
February 05, 2023
None
BLACKMAGIC DESIGN POCKET CINEMA CAMERA 4K BLACK MAGIC BMPCC4K 12-BIT BRAW
1269.99
January 23, 2023
None
BLACKMAGIC DESIGN POCKET CINEMA CAMERA 4K BLACK MAGIC BMPCC4K 12-BIT BRAW
1269.99
January 20, 2023
None
Blackmagic Design Pocket Cinema Camera 4K, 12 Monate Restgarantie!
899.0
January 19, 2023
None
Blackmagic Pocket Cinema Camera 4K Zubehörpaket bester Zustand
800.0
January 12, 2023
None
Blackmagic Pocket Cinema Camera 4K Blackmagicdesign   Neu
1299.0
January 10, 2023
None
BLACKMAGIC DESIGN POCKET CINEMA CAMERA 4K BLACK MAGIC BMPCC4K 12-BIT BRAW
1249.99
January 08, 2023
None
Black Magic Pocket Cinema Camera 6K Pro SmallRig Feelworld 4K Monitor
1787.55
January 04, 2023
None
blackmagic design pocket cinema camera 4k
870.0
January 02, 2023
None
Blackmagic Pocket Cinema Cam

In [2]:
# Same as previous cell, but with integration of pandas in mind:
def convertResultsToPandas(page_source):
    soup = BeautifulSoup(page_source)
    results = soup.find("ul", {"class" : "srp-results srp-list clearfix"})
    # Create empty list (or numpy array) for storing the rows of df
    rows = []
    items = results.findAll("li")
    for item in items:
        if "Ergebnisse für weniger Suchbegriffe" in item.strings:
            break
        itemTitle = item.find("div", {"class" : "s-item__title"})
        itemPrice = item.find("span", {"class": "s-item__price"})
        itemDate = item.find("span", {"class" : "POSITIVE"}) #POSITIVE=sold, NEGATIVE=ended (no bids)
        # There are some itemTitle None. Exclude them
        if itemTitle and itemPrice and itemDate:
            # Delete the annoying "NEW LISTING" Tag:
            newListingTag = itemTitle.find("span", {"class" : "LIGHT_HIGHLIGHT"})
            if newListingTag:
                newListingTag.extract()
            itemTitle = itemTitle.text
            #print(itemTitle)
            
            # Transform the price string to float
            itemPrice = itemPrice.text
            itemPrice = float(itemPrice.split()[-1].replace(".", "").replace(",","."))
            #print(itemPrice)
    
            # Eliminate "verkauft" from the date
            itemDate = itemDate.text
            itemDate = itemDate.split()[1:]
            # HARDCODED: Change Dez to Dec
            itemDate[1] = "Dec" if itemDate[1] == "Dez" else itemDate[1]
            # Apply date formatting %b %d, %Y
            itemDate = itemDate[1]+" "+itemDate[0][:-1]+", "+itemDate[2]
            # Create a datetime object to store the date
            itemDate = datetime.strptime(itemDate, "%b %d, %Y")
            #print(itemDate.strftime("%B %d, %Y"))
            
            # Now create a row for the df from the extracted data
            row = [itemTitle, itemPrice, itemDate]
            rows.append(row)
    df = pd.DataFrame.from_records(rows, columns=["Title", "Price [€]", "Purchase Date"])
    return df   


In [6]:
# Now we would like to iterate through the result pages of Ebay.
def convertEbaySearchtoPandas_deprecated_(url, gdprBannerAccepted=False):
    driver.get(url)
    if not gdprBannerAccepted:
        time.sleep(3)
        driver.find_element(By.ID, "gdpr-banner-accept").click()
        gdprBannerAccepted = True
    
    df_total = pd.DataFrame(columns=["Title", "Price [€]", "Purchase Date"])
    count = 0
    while(True):
        if count == 3:
            break
        df = convertResultsToPandas(driver.page_source)
        #print(df)
        df_total = pd.concat([df_total, df])
        
        # Find the "NextPage" button:
        nextPageLink = driver.find_element(By.CLASS_NAME, "pagination__next") #Xpath: "//a[@class='pagination__next icon-link']"
        # Get the attribute abled/disabled:
        isDisabledNextPage = nextPageLink.get_attribute("aria-disabled") == "true"
        
        if isDisabledNextPage:
            break
        else:
            nextPageLink.click()
            time.sleep(2)
            count+=1
            
    print(df_total)

In [4]:
driver = webdriver.Firefox()
driver.implicitly_wait(10)

In [32]:
url = "https://www.ebay.de/sch/i.html?_dcat=27386&_fsrp=1&_from=R40&_nkw=rtx+3070&_sacat=0&LH_TitleDesc=0&Speichergr%25C3%25B6%25C3%259Fe=8%2520GB&rt=nc&LH_Sold=1&LH_Complete=1" 
convertEbaySearchtoPandas(url, True)

                                                Title  Price [€] Purchase Date
0   Inno3D GeForce RTX 3070 TWIN X2 OC 8GB GDDR6 G...     400.00    2023-02-20
1               MSI GeForce RTX 3070 VENTUS 3X OC 8GB     450.00    2023-02-20
2   ASUS NVIDIA GeForce RTX 3070 TUF Gaming OC 8GB...     405.00    2023-02-20
3   ASUS ROG Strix GeForce RTX 3070 OC 8GB GDDR6 G...     450.00    2023-02-20
4   Gigabyte AORUS GeForce RTX 3070 Ti Master 8GB ...     749.00    2023-02-20
..                                                ...        ...           ...
55  GIGABYTE GeForce RTX 3070 Gaming OC R2.0 8GB G...     616.48    2023-02-04
56  GIGABYTE GeForce RTX 3070 VISION OC 8‎GB GDDR6...     440.00    2023-02-04
57  ASUS TUF GeForce RTX 3070 TI 8GB Gaming Grafik...     699.90    2023-02-04
58  ASUS TUF GeForce RTX 3070 TI 8GB OC Version Ga...     699.90    2023-02-04
59  ASUS TUF GeForce RTX 3070 V2 8 GB OC Version G...     649.90    2023-02-04

[180 rows x 3 columns]


In [5]:
# If we automate webdriver operation in the function we can omitt the condition:

def convertEbaySearchtoPandas(url):
    # Webdriver operation: open Browser, go to Url, accept Ebay Banner
    driver = webdriver.Firefox()
    driver.implicitly_wait(10)
    driver.get(url)
    time.sleep(3)
    driver.find_element(By.ID, "gdpr-banner-accept").click()
    
    # Initialize empty Data Frame
    df_total = pd.DataFrame(columns=["Title", "Price [€]", "Purchase Date"])

    while(True):
        df = convertResultsToPandas(driver.page_source)
        #print(df)
        df_total = pd.concat([df_total, df])
        
        # HARDCODED: set isDisabledNextPage to True for the case an Exception is raised
        isDisabledNextPage = True
        try:
            # Find the "NextPage" button
            nextPageLink = driver.find_element(By.CLASS_NAME, "pagination__next") #Xpath: "//a[@class='pagination__next icon-link']"
            # Get the attribute abled/disabled
            isDisabledNextPage = nextPageLink.get_attribute("aria-disabled") == "true"
        except NoSuchElementException:
            print("Results consist of only 1 page. Exiting...\n")
        
        if isDisabledNextPage:
            break
        else:
            nextPageLink.click()
            time.sleep(2)
    
    driver.close()
    
    return df_total

In [16]:
url = "https://www.ebay.de/sch/i.html?_dcat=27386&_fsrp=1&_from=R40&_nkw=rtx+3070&_sacat=0&LH_TitleDesc=0&Speichergr%25C3%25B6%25C3%259Fe=8%2520GB&rt=nc&LH_Sold=1&LH_Complete=1" 
df_rtx_3060 = convertEbaySearchtoPandas(url)


In [18]:
df_rtx_3060 = df_rtx_3060.assign(Model = "RTX 3070")
df_rtx_3060

,Title,Price [€],Purchase Date,Model
0,Lenovo OEM Nvidia GeForce RTX 3070 8GB GDDR6 G...,399.0,2023-02-21,RTX 3070
1,ZOTAC GeForce RTX 3070 AMP Holo 8GB GDDR6 Graf...,499.0,2023-02-21,RTX 3070
2,ASUS ROG Strix GeForce RTX 3070 OC White Editi...,499.0,2023-02-21,RTX 3070
3,ASUS GeForce RTX 3070 DUAL OC 8GB GDDR6 Grafik...,425.0,2023-02-20,RTX 3070
4,NVIDIA GeForce RTX 3070 Founders Edition 8GB G...,419.0,2023-02-20,RTX 3070
...,...,...,...,...
43,"ASUS Dual GeForce RTX 3070 OC, DUAL-RTX3070-O8...",429.0,2022-11-23,RTX 3070
44,ASUS TUF Gaming GeForce RTX 3070 OC 8GB GDDR6...,440.0,2022-11-23,RTX 3070
45,GIGABYTE AORUS GeForce RTX 3070 8GB GDDR6 Graf...,470.0,2022-11-23,RTX 3070
46,[NEU | Bundle] ASUS Dual GeForce RTX 3070 V2 8...,555.0,2022-11-23,RTX 3070


In [14]:
url = "https://www.ebay.de/sch/i.html?_from=R40&_nkw=blackmagic+pocket+cinema+camera+4k&_sacat=0&LH_TitleDesc=0&_fsrp=1&LH_Complete=1&LH_Sold=1&Marke=Blackmagic%7CBlackmagic%2520Design&_dcat=11724&rt=nc&LH_PrefLoc=3"
df_bmpcc_4k = convertEbaySearchtoPandas(url)

Results consist of only 1 page. Exiting...



In [15]:
df_bmpcc_4k = df_bmpcc_4k.assign(Model = "BMPCC 4K")
df_bmpcc_4k

,Title,Price [€],Purchase Date,Model
0,BLACKMAGIC DESIGN POCKET CINEMA CAMERA 4K BLAC...,942.00,2023-02-08,BMPCC 4K
1,Blackmagic Design 4K 120fps Kamera - SET,1050.00,2023-02-05,BMPCC 4K
2,BLACKMAGIC DESIGN POCKET CINEMA CAMERA 4K BLAC...,1249.99,2023-01-23,BMPCC 4K
3,BLACKMAGIC DESIGN POCKET CINEMA CAMERA 4K BLAC...,1269.99,2023-01-20,BMPCC 4K
4,"Blackmagic Design Pocket Cinema Camera 4K, 12 ...",899.00,2023-01-19,BMPCC 4K
5,Blackmagic Pocket Cinema Camera 4K Zubehörpake...,800.00,2023-01-12,BMPCC 4K
6,Blackmagic Pocket Cinema Camera 4K Blackmagicd...,1299.00,2023-01-10,BMPCC 4K
7,BLACKMAGIC DESIGN POCKET CINEMA CAMERA 4K BLAC...,1249.99,2023-01-08,BMPCC 4K
8,Black Magic Pocket Cinema Camera 6K Pro SmallR...,1787.55,2023-01-04,BMPCC 4K
9,blackmagic design pocket cinema camera 4k,870.00,2023-01-02,BMPCC 4K


In [7]:
url = "https://www.ebay.de/sch/i.html?_from=R40&_nkw=nikon+F2&_sacat=0&LH_Sold=1&LH_Complete=1&rt=nc&Modell=Nikon%2520F2&_dcat=15230"
df_nikon_F2 = convertEbaySearchtoPandas(url)

Results consist of only 1 page. Exiting...



In [8]:
#df_nikon_F2 = df_nikon_F2.assign(Model = "Nikon F2")
df_nikon_F2.to_csv()

,Title,Price [€],Purchase Date,Model
0,*AS-IS* Nikon F2 Body Black 35mm SLR,98.08,2023-02-19,Nikon F2
1,Nikon f 2 defekt mit Original Nikkor Objektiv,134.77,2023-02-15,Nikon F2
2,Nikon F2 analoge Spiegelreflexkamera Gewährlei...,245.00,2023-02-14,Nikon F2
3,Nikon F2 AS Photomic SLR 35mm Gehäuse Body Sil...,335.00,2023-02-12,Nikon F2
4,Nikon F2 AS SLR Spiegelreflexkamera Body Gehäuse,288.00,2023-02-12,Nikon F2
5,Nikon F2 H-MD Titan kompl. Set NOS unbenutzt n...,5550.00,2023-02-10,Nikon F2
6,Nikon F2 Photomic schwarz + Motordrive MD-2 + ...,299.99,2023-02-05,Nikon F2
7,Nikon F2 35mm Spiegelreflexkamera Body mit Ri...,350.00,2023-02-05,Nikon F2
8,nikon f2 as,301.00,2023-02-04,Nikon F2
9,Nikon F2 + DP1 Photomic Camera Body Chrome Geh...,249.99,2023-02-03,Nikon F2


In [19]:
df_nikon_F2.to_csv("nikon_F2.csv")
df_bmpcc_4k.to_csv("bmpcc_4k.csv")
df_rtx_3060.to_csv("RTX_3070.csv")